In [19]:
import wrds
import pandas as pd
import os
import json
from datetime import datetime
import re

In [20]:
# set the directory path
directory = "articles"

# initialize an empty list to store the data
data = []

# loop through each directory in the Articles directory
for subdir in os.listdir(directory):
    subdir_path = os.path.join(directory, subdir)
    if os.path.isdir(subdir_path):
        # loop through each file in the directory
        for filename in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, filename)
            if os.path.isfile(file_path) and filename.endswith(".json"):
                # read the JSON file
                with open(file_path, "r") as f:
                    json_obj = json.load(f)
                # extract the relevant information from the JSON object
                _id = json_obj["_id"]
                #name of the directory is the company name
                company = subdir
                title = json_obj["title"]
                text = json_obj["text"]
                cleaned_text = re.sub(r'<p>|</p>', '', text)
                #Change to timestamp YYYY-MM-DD HH:MM:SS
                published = json_obj["published"]
                timestamp = datetime.strptime(published, '%Y-%m-%dT%H:%M:%S.%fZ')
                data.append([_id, company, title, cleaned_text, timestamp])
        
df = pd.DataFrame(data, columns=["id", "company", "title", "text", "published"])

In [21]:
#Remove duplicates texts
df = df.sort_values(['company', 'published'])
df = df.drop_duplicates(subset=['company','text'], keep='first')

In [22]:
#print company names
company_to_ticker = {
    '3M_Company': 'MMM',
    'American_Express_co': 'AXP',
    'Amgen_Inc': 'AMGN',
    'Apple_Inc': 'AAPL',
    'Boeing_Co': 'BA',
    'Caterpillar_Inc': 'CAT',
    'Chevron_Corporation': 'CVX',
    'Cisco_Systems_Inc': 'CSCO',
    'Coca_Cola_Co': 'KO',
    'Dow_Inc': 'DOW',
    'Goldman_Sachs_Group_Inc': 'GS',
    'Home_Depot_Inc': 'HD',
    'Honeywell_International_Inc': 'HON',
    'Intel_Corporation': 'INTC',
    'International_Business_Machines_Corporation': 'IBM',
    'JPMorgan_Chase_Co': 'JPM',
    'Johnson_Johnson': 'JNJ',
    'McDonald_s_Corporation': 'MCD',
    'Merck_Co_Inc': 'MRK',
    'Microsoft_Corporation': 'MSFT',
    'Nike_Inc': 'NKE',
    'Procter_Gamble_Co': 'PG',
    'Salesforce_Inc': 'CRM',
    'The_Walt_Disney_Company': 'DIS',
    'Travelers_Companies_Inc': 'TRV',
    'Unitedhealth_Group_Incorporated': 'UNH',
    'Verizon_communications_Inc': 'VZ',
    'Visa_Inc': 'V',
    'Walgreens_Boots_Alliance_Inc': 'WBA',
    'Walmart_Inc': 'WMT'
}

#Replace company names with ticker symbols
df['ticker'] = df['company'].map(company_to_ticker)

In [23]:
#Helpers functions for cleaning text
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def remove_ftcom_and_after(text):
    index = text.find("FT.com")
    return text[:index] if index != -1 else text

def remove_source_and_after(text) : 
    index  = text.find("Source:")
    return text[:index] if index != -1 else text

def replace_PG(text):
    return text.replace("P&amp;G", "Procter & Gamble")

In [24]:
financial_times_news_df = df[df['text'].str.contains('Financial Times')]
financial_times_news_df.text.values[0]

'For cold-blooded markets, the big political event of the week was not so much the lawless mob storming the <a href="https://www.ft.com/content/9a37c259-f7c8-4cca-9a00-83e7e5b9287d">US Capitol building</a>. Individual investors were shocked and repulsed by the events in Washington spurred by the rages of Donald Trump in the final days of his tumultuous presidency. But after a brief wobble on Wednesday, US and global equities resumed their recent trend of setting all-time highs.  Investors appeared to be looking ahead to the transfer of power that culminates in the inauguration of Joe Biden as US president this month — a succession belatedly <a href="https://www.ft.com/content/15f932cf-b8e5-4c24-b0d5-1ef903103a3c">acknowledged</a> by Mr Trump. “Markets, rightly in our view, see the US government as ultimately a stable enough set of institutions even if things occasionally go pear-shaped,” said Nicholas Colas, co-founder of independent research house DataTrek.  Of greater significance to

In [25]:
#Remove html tags
df['text'] = df['text'].apply(remove_html_tags)

#Remove FT.com and after
df['text'] = df['text'].apply(remove_ftcom_and_after)

#Remove Source: and after
df['text'] = df['text'].apply(remove_source_and_after)

#Change P&G tokens 
df['text'] = df['text'].apply(replace_PG)

#Remove &#xa0; and replace with space
df['text'] = df['text'].str.replace('&#xa0;', ' ')

#Remove "Follow @FT" and after
df['text'] = df['text'].apply(lambda x: x.split("Follow @FT")[0])

#Remove rows with "Please sign up here
df = df[df['text'] != "Please sign up here"]

#Keep rows that contain "FT"
#df = df[df['text'].str.contains('FT')]
df

,id,company,title,text,published,ticker
182988,1ab4eb2837fe066da76fb56cc16f4ea7ecdd529f633c9e...,3M_Company,Investors look past the storming of US Capitol,"For cold-blooded markets, the big political ev...",2021-01-09 05:00:26.214,MMM
182414,2dd60e56c3c327788dc2db6973febb45f6649144893028...,3M_Company,EXCHANGE --- Business News: Firms Team Up on V...,The agreement is part of the federal governmen...,2021-01-16 00:00:00.000,MMM
182603,64e78c0e9fec3152693f37c0244fd7f56cb30c0cab52d5...,3M_Company,EXCHANGE --- Business News: Firms Team Up o...,Medical-gear makers are sharing production ...,2021-01-16 07:07:00.000,MMM
182462,3ee50b6ac8c67d2f5587c3ea6b7a554dff9c96f202041c...,3M_Company,"Pandemic price rises still rampant on Amazon, ...",Hundreds of the essential products that have c...,2021-01-22 04:00:11.785,MMM
182137,1f29e59d64136d04f00309aae496d6b4d3c6ddba725b7a...,3M_Company,"Vietnam congress, Fed meeting, tech reports, D...",This week brings two big gatherings with the D...,2021-01-24 17:16:44.025,MMM
...,...,...,...,...,...,...
12317,3544723c86785e7d2ed47a0c0038c2549395732fc71301...,Walmart_Inc,From a Deepfaked Elon Musk to RadioShack's NSF...,Other widely read articles by CMO Today covere...,2022-12-29 21:33:00.000,WMT
14073,0c92e3845275cf0102660bd04bbf29060d8311a4037f8d...,Walmart_Inc,The Shoplifting Tax Keeps Going Up; When Targe...,Rite Aid said in an earnings callthis month th...,2022-12-29 23:27:00.000,WMT
5808,694b3271af75f7ba40958e1d0e2262651b743f8996d72d...,Walmart_Inc,George Santos Loses Some GOP Support on Long I...,Nassau County Republican Chairman Joe Cairo sa...,2022-12-30 22:50:00.000,WMT
10463,cddb247a84fa1c71046c49eca567266fe7b847f4eedfa9...,Walmart_Inc,U.S. News: Republicans in Home District Seek D...,Nassau County Republican Chairman Joe Cairo sa...,2022-12-31 00:00:00.000,WMT


In [26]:
# from difflib import SequenceMatcher

# def similar_words(text, target, similarity_threshold=0.8):
#     # Split the text into words
#     words = text.split()

#     # Find words that are similar to the target
#     similar = [word for word in words if SequenceMatcher(None, word, target).ratio() > similarity_threshold]

#     return similar

# df['similar_words'] = df.apply(lambda row: similar_words(row['text'], row['company']), axis=1)

#They are going to give us the list of words that are similar to the company name

In [27]:
ticker_to_company = {'MMM' : '3m',
                     'AXP' : 'american express',
                     'AMGN' : 'amgen',
                     'AAPL' : 'apple',
                     'BA' : 'boeing',
                     'CAT' : 'caterpillar',
                     'CVX' : 'chevron',
                     'CSCO' : 'cisco',
                     'KO' : 'coca cola',
                     'DOW' : 'dow',
                     'GS' : 'goldman sachs',
                     'HD' : 'home depot',
                     'HON' : 'honeywell',
                     'INTC' : 'intel',
                     'IBM' : 'ibm',
                     'JPM' : 'jpmorgan',
                     'JNJ' : 'johnson & johnson',
                     'MCD' : 'mcdonalds',
                     'MRK' : 'merck',
                     'MSFT' : 'microsoft',
                     'NKE' : 'nike',
                     'PG' : 'procter & gamble',
                     'CRM' : 'salesforce',
                     'DIS' : 'walt disney',
                     'TRV' : 'travelers',
                     'UNH' : 'united health',
                     'VZ' : 'verizon',
                     'V' : 'visa',
                     'WBA' : 'walgreens',
                     'WMT' : 'walmart'}

df['company_words'] = df['ticker'].map(ticker_to_company)

In [28]:
df['text'] = df['text'].str.lower()
df['sentences'] = df.apply(lambda row: [sentence for sentence in row['text'].split('. ') if row['company_words'] in sentence], axis=1)

In [31]:
#For delete the sentences that contain Listen on:
df['sentences'] = df.apply(lambda row: [sentence for sentence in row['sentences'] if 'listen on:' not in sentence], axis=1)

In [32]:
#Save the dataframe to a csv file
df.to_csv('data/news_cleaned.csv', index=False)